In [1]:
import numpy as np
import cv2
import sys
import os

def retinex(img, sigma_list, G, b, alpha, beta, low_clip, high_clip):

    def singleScaleRetinex(img, sigma):
        retinex = np.log10(img) - np.log10(cv2.GaussianBlur(img, (0, 0), sigma))

        return retinex


    def multiScaleRetinex(img, sigma_list):
        retinex = np.zeros_like(img)   
        for sigma in sigma_list:
            retinex += singleScaleRetinex(img, sigma)  

        retinex = retinex / len(sigma_list)

        return retinex


    def colorRestoration(img, alpha, beta):
        img_sum = np.sum(img, axis=2, keepdims=True)

        color_restoration = beta * (np.log10(alpha * img) - np.log10(img_sum))

        return color_restoration


    def simplestColorBalance(img, low_clip, high_clip):
        total = img.shape[0] * img.shape[1]
        for i in range(img.shape[2]):
            unique, counts = np.unique(img[:, :, i], return_counts=True)
            current = 0
            for u, c in zip(unique, counts):
                if float(current) / total < low_clip:
                    low_val = u
                if float(current) / total < high_clip:
                    high_val = u
                current += c

            img[:, :, i] = np.maximum(np.minimum(img[:, :, i], high_val), low_val)

        return img


    def MSRCR(img, sigma_list, G, b, alpha, beta, low_clip, high_clip):
        img = np.float64(img) + 1.0

        img_retinex = multiScaleRetinex(img, sigma_list)

        img_color = colorRestoration(img, alpha, beta)
        img_msrcr = G * (img_retinex * img_color + b)

        for i in range(img_msrcr.shape[2]):
            img_msrcr[:, :, i] = (img_msrcr[:, :, i] - np.min(img_msrcr[:, :, i])) / \
                                 (np.max(img_msrcr[:, :, i]) - np.min(img_msrcr[:, :, i])) * \
                                 255

        img_msrcr = np.uint8(np.minimum(np.maximum(img_msrcr, 0), 255))
        img_msrcr = simplestColorBalance(img_msrcr, low_clip, high_clip)

        return img_msrcr
    
    retinex = MSRCR(img, sigma_list, G, b, alpha, beta, low_clip, high_clip)
    return retinex



In [3]:
data_path = "/Users/jiangshihua/Desktop/1/test/e0/"
img_list = os.listdir(data_path)
if len(img_list) == 0:
    print('Data directory is empty.')
    exit()


for img_name in img_list:
    if img_name == '.gitkeep':
        continue

    img = cv2.imread(os.path.join(data_path, img_name))

    sigma_list = [15, 80, 200]
    G = 5.0
    b = 25.0
    alpha = 125.0
    beta = 46.0
    low_clip = 0.01
    high_clip = 0.99
   
    print('msrcr processing......')
    a = retinex(img, sigma_list, G, b, alpha, beta, low_clip, high_clip)

    #cv2.imshow('MSRCR retinex', a)
    cv2.imwrite("MSRCR_"+img_name+".tif", a);

    #cv2.waitKey()


msrcr processing......
msrcr processing......


ValueError: non-broadcastable output operand with shape () doesn't match the broadcast shape (4,1)